In [2]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
 
# Create a new chat bot named Charlie
chatbot = ChatBot('Charlie')
 
trainer = ListTrainer(chatbot)
 
trainer.train([
    "你好",
    "你是谁",
    "你好啊"
])
 
# Get a response to the input text 'I would like to book a flight.'
response = chatbot.get_response('你是谁')
 
print(response)
 
 

List Trainer: [####################] 100%
你好啊


In [1]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
 
# 构建ChatBot并指定
bot = ChatBot(
    'Default Response Example Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch'
        },
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'threshold': 0.6,
            'default_response': 'I am sorry, but I do not understand.'
        }
    ],
    trainer='chatterbot.trainers.ListTrainer'
)
bot2=ListTrainer(bot)
# 手动给定一点语料用于训练
bot2.train([
   'How can I help you?',
   'I want to create a chat bot',
   'Have you read the documentation?',
   'No, I have not',
   'This should help get you started: http://chatterbot.rtfd.org/en/latest/quickstart.html',
    'fuck you',
    '去你的，好好说话'
])
# 给定问题并取回结果
# question = '你好'
# print(question)
# response = bot.get_response(question)
# print(response)
# print("\n")
# question = '你是谁'
# print(question)
# response = bot.get_response(question)
# print(response)

while True:
    xx=input()
    response=bot.get_response(xx)
    print(response)

List Trainer: [####################] 100%


KeyboardInterrupt: Interrupted by user

In [ ]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

chatbot = ChatBot("ChineseChatBot")

trainer = ChatterBotCorpusTrainer(chatbot)
# 使用中文语料库训练它
trainer.train("chatterbot.corpus.chinese")

# 开始对话
while True:
    print(chatbot.get_response(input(">")))


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%
>你好
你是谁
>我是你爸爸
水的别称
>我是你爹
继续，问吧.
>你叫什么名字
即便假借特例的实用性之名,
>我叫张润民
什么是股市
>你搞什么啊
No, I have not
>你会不会打篮球啊
电力
>你好笨啊
你永远无法真正预测股市。
>橘子
不错
>西瓜
你如何运作?
>你真笨
你是没

In [1]:
# redis 测试
import redis
from wxpy import *
# init redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
# init bot
bot = Bot()
# info store in redis as list
def group_info_store(info):
    r.rpush('group_info',info)
# export group info
def group_info_export():
    size=r.llen("group_info") 
    for i in range(0,size):
        print(r.lindex("group_info",i))
# delete group info
def remove_group_info():
    size=r.llen("group_info") 
    for i in range(0,size):
        r.brpop('group_info')

# deal info with group   
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
#     msg.chat.send(msg.text)
    username=msg.member.name
    info=msg.text
    user_info=str(username)+":"+str(info)
#     print(user_info)
    
    group_info_store(user_info)
    
    
    if(info=='导出聊天记录'):
        print('----导出中----')
        group_info_export()
        print('----导出完成----')
    elif(info=='删除聊天记录'):
        print('----删除中----')
        remove_group_info()
        print('----删除完成----')
    
    
    

    
    

    
@bot.register(Friend,TEXT)
def auto_accept_friends(msg):
    msg.chat.send(msg.text)


Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 测试用的


ZRM:@
ZRM:导出聊天记录
----导出中----
ZRM:导出聊天记录
ZRM:1
-Y:??????
ZRM:@
ZRM:导出聊天记录
ZRM:在测试
ZRM:导出聊天记录
----导出中----
ZRM:导出聊天记录
ZRM:1
-Y:??????
ZRM:@
ZRM:导出聊天记录
ZRM:在测试
ZRM:导出聊天记录
ZRM:删除聊天记录
----删除完成----
ZRM:导出聊天记录
----导出中----
ZRM:导出聊天记录
ZRM:[皱眉]


In [2]:
bot.self.name

'测试用的'